<!-- New Markdown Cell: Model Selection & ΔSNR Summary -->

# 🧪 Checkpoint Model Selection & ΔSNR Assessment

### 🎯 Goal  
Identify which checkpoint is most (over)fit to the `krishna/` dataset while also computing the average ΔSNR across a broader `general/` dataset.

---

### 📂 Data Layout (assumed)
- `checkpoints/` → multiple model folders or files (e.g. `epoch_*.pt` / `model_*.ckpt`)
- `data/krishna/` → targeted evaluation set (possible overfitting detection)
- `data/general/` → broader evaluation set

---

### ✅ Metrics
| Metric | Krishna | General | Notes |
|--------|---------|---------|-------|
| Loss (val) | ... | ... | Lower may indicate overfit if gap large |
| SNR_raw | ... | ... | Baseline signal-to-noise |
| SNR_model | ... | ... | After model enhancement |
| ΔSNR = SNR_model − SNR_raw | ... | ... | Use mean over samples |
| ΔSNR (avg) | ... | ... | Compare across checkpoints |

---

### 🧮 ΔSNR Computation
For each sample i:  
ΔSNRᵢ = SNR_modelᵢ − SNR_rawᵢ  
Aggregate: mean(ΔSNRᵢ), std(ΔSNRᵢ)

---

### 🔍 Overfitting Heuristics
- Very low Krishna loss + relatively poorer General ΔSNR → likely overfit.
- Large gap: (Krishna ΔSNR − General ΔSNR) → potential over-specialization.
- Checkpoint preference: maximize General ΔSNR while keeping Krishna performance acceptable.

---

### 🪜 Procedure
1. Enumerate checkpoints.
2. For each: load model, run inference on Krishna & General datasets.
3. Compute raw SNR (pre-enhancement) and enhanced SNR.
4. Store:
   - val_loss (if available)
   - mean ΔSNR (Krishna)
   - mean ΔSNR (General)
5. Rank primarily by General ΔSNR; flag overfit if Krishna >> General.
6. Select: Highest General ΔSNR with smallest Krishna−General gap (or according to project goals).

---

### 📊 Output Suggestion
Produce a sorted table:
Checkpoint | val_loss | ΔSNR_Krishna | ΔSNR_General | Gap | Overfit_Flag

---

### 🧷 Notes
- Ensure consistent SNR formula: SNR = μ_signal / σ_noise (or 20·log10 if amplitude-based).
- Normalize input if models expect standardized tensors.
- Use same random seed for reproducibility.
- Optionally add violin plots for ΔSNR distributions.

---

### 🛠 Next Cell: Runner
A Python cell will implement this evaluation loop.

#### Depending upon the particular overfitting of the data I might have to use multiple models for multiple spectra..... if possible

In [ ]:
# Model checkpoint evaluation and ΔSNR computation.

import os
import math
import torch
import numpy as np
from typing import Dict, List, Tuple
from pathlib import Path
from statistics import mean, stdev

# ...existing code...
CHECKPOINT_DIR = Path("checkpoints")
KRISHNA_DIR = Path("data/krishna")
GENERAL_DIR = Path("data/general")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def list_checkpoints(path: Path) -> List[Path]:
    return sorted([p for p in path.iterdir() if p.suffix in (".pt", ".ckpt")])

def load_model(cp: Path):
    model = torch.load(cp, map_location=DEVICE)
    if hasattr(model, 'eval'):
        model.eval()
    return model

def load_dataset(folder: Path) -> List[torch.Tensor]:
    tensors = []
    for f in sorted(folder.glob("*.npy")):
        arr = np.load(f)
        tensors.append(torch.tensor(arr, dtype=torch.float32, device=DEVICE))
    return tensors

def compute_snr(raw: torch.Tensor, enhanced: torch.Tensor) -> Tuple[float,float,float]:
    # Simple SNR: mean(signal)/std(noise_residual)
    signal_mean = raw.abs().mean().item()
    noise_residual = (raw - enhanced)
    noise_std = noise_residual.std().item() + 1e-12
    snr_raw = signal_mean / (raw.std().item() + 1e-12)
    snr_enh = signal_mean / noise_std
    return snr_raw, snr_enh, snr_enh - snr_raw

def evaluate_checkpoint(model, data: List[torch.Tensor]) -> Dict[str, float]:
    deltas = []
    for x in data:
        with torch.no_grad():
            y = model(x.unsqueeze(0))
            if isinstance(y, (list, tuple)):
                y = y[0]
            y = y.squeeze(0)
        snr_raw, snr_enh, delta = compute_snr(x, y)
        deltas.append(delta)
    return {
        "delta_mean": mean(deltas),
        "delta_std": stdev(deltas) if len(deltas) > 1 else 0.0
    }

def try_val_loss(model) -> float:
    return getattr(model, "val_loss", float("nan"))

def main():
    checkpoints = list_checkpoints(CHECKPOINT_DIR)
    krishna_data = load_dataset(KRISHNA_DIR)
    general_data = load_dataset(GENERAL_DIR)

    rows = []
    for cp in checkpoints:
        model = load_model(cp)
        kr = evaluate_checkpoint(model, krishna_data)
        ge = evaluate_checkpoint(model, general_data)
        val_loss = try_val_loss(model)
        gap = kr["delta_mean"] - ge["delta_mean"]
        rows.append({
            "checkpoint": cp.name,
            "val_loss": val_loss,
            "delta_krishna": kr["delta_mean"],
            "delta_general": ge["delta_mean"],
            "gap": gap
        })

    # Ranking by general ΔSNR descending
    rows.sort(key=lambda r: r["delta_general"], reverse=True)

    print(f"{'Checkpoint':30} {'val_loss':>10} {'ΔSNR_K':>10} {'ΔSNR_G':>10} {'Gap':>10}")
    for r in rows:
        print(f"{r['checkpoint']:30} {r['val_loss']:10.4f} {r['delta_krishna']:10.4f} {r['delta_general']:10.4f} {r['gap']:10.4f}")

    best = rows[0] if rows else None
    if best:
        print("\nSelected (tentative) best (general ΔSNR):", best['checkpoint'])

if __name__ == "__main__":
    main()
# ...existing code...